In [ ]:
import xarray as xr
import numpy as np
import dask
import dask.array as da
from dask.diagnostics import ProgressBar

In [ ]:
from linearsim import time_domain_das

# Generate a dataset of input spectra 

In [ ]:
ds_spec = xr.Dataset()
ds_spec['Gamma'] = xr.DataArray(da.arange(1,7,0.5),dims='gamma')
ds_spec['Tp'] = xr.DataArray(da.arange(8,20.5,0.5),dims='tp')
ds_spec['Duration'] = xr.DataArray(da.array([2,5,10,20,40,60])*60,dims='duration')
n_seed = 10000
ds_spec['Seed'] = xr.DataArray(da.arange(n_seed,chunks=(500,)).astype('int'),dims='seed')
ds_spec['this_seed'] = xr.DataArray(da.random.choice(int(1E9), size=(len(ds_spec['Tp']), len(ds_spec['Gamma']), len(ds_spec['Duration']), n_seed)).astype('int'), dims=('tp','gamma','duration','seed'))
ds_spec = ds_spec.chunk({'tp':1,'gamma':-1,'duration':1,'seed':2000})
ds_spec

In [ ]:
ds_spec['Tz'], ds_spec['Hs'], ds_spec['Hmax'], ds_spec['H13'], ds_spec['r'], _ = xr.apply_ufunc(time_domain_das,
                                                                                ds_spec['Tp'],
                                                                                1.0,
                                                                                ds_spec['Gamma'],
                                                                                ds_spec['Duration'],
                                                                                0.5,
                                                                                ds_spec['this_seed'],
                                                                                kwargs=dict(fft_equiv_duration=60*60),
                                                                                input_core_dims=[[],[],[],[],[],[]],
                                                                                output_core_dims=[[],[],[],[],[],[]],
                                                                                vectorize=True,
                                                                                dask='parallelized',
                                                                                output_dtypes=['float','float','float','float','float','int']
                                                                                )

In [ ]:
ds_spec_chunked = ds_spec.chunk({'tp':1,'gamma':-1,'duration':1,'seed':-1})

# Use multiple processes to calculate timeseries
with dask.config.set(scheduler='processes'):
    with ProgressBar():
        ds_spec_chunked.to_zarr('data/timeseries_stats/DAS_n10000_2Hz_fixedres.zarr',mode='w')

# Analyse saved results

In [ ]:
ds_spec = xr.open_zarr('DAS_n10000_2Hz_fixedres.zarr').load()
ds_spec = ds_spec.rename({'Duration':'duration','Gamma':'gamma','Tp':'tp','Seed':'seed'})
ds_spec = ds_spec.assign_coords({'duration':ds_spec['duration'],'gamma':ds_spec['gamma'],'tp':ds_spec['tp'],'seed':range(len(ds_spec['seed']))})
ds_spec['HmHs'] = ds_spec['Hmax']/ds_spec['Hs']
ds_std = ds_spec.std(dim='seed')
ds_mean = ds_spec.mean(dim='seed')
ds_max = ds_spec.max(dim='seed')
ds_CV = ds_std/ds_mean
ds_CV

In [ ]:
import hvplot.xarray
import holoviews as hv

In [ ]:
layout = []
for v in ds_mean.data_vars:
    layout.append(ds_mean[v].hvplot.image(groupby=['duration'],cmap='viridis',aspect=1,title=v))
layout = hv.Layout(layout).cols(3)

hv.save(layout,filename="figures/timeseries_stats/DAS_Mean_fixedres.html")

In [ ]:
layout = []
for v in ds_CV.data_vars:
    layout.append(ds_CV[v].hvplot.image(groupby=['duration'],cmap='viridis',aspect=1,title=v))
layout = hv.Layout(layout).cols(3)

hv.save(layout,filename="figures/timeseries_stats/DAS_CV_fixedres.html")

In [ ]:
layout = []
for v in ds_max.data_vars:
    layout.append(ds_max[v].hvplot.image(groupby=['duration'],cmap='viridis',aspect=1,title=v))
layout = hv.Layout(layout).cols(3)

hv.save(layout,filename="figures/timeseries_stats/DAS_Max_fixedres.html")

## Take a look at the largest wave event

In [ ]:
hv.extension('bokeh')

In [ ]:
ds_stacked = ds_spec.stack(sample=['tp','duration','gamma','seed'])
im=ds_stacked.Hmax.argmax()
largest_wave = ds_stacked.isel(sample=im)

display(largest_wave)

tp, duration, gamma,_ = np.array(largest_wave.sample.values.item())
seed = int(largest_wave.this_seed)
dt=0.5
ts=time_domain_das(tp,1.0,gamma,duration,dt=dt,seed=seed,return_ts=True)
da_ts = xr.DataArray(ts,dims='time',coords={'time':np.arange(0,duration,dt)},name='timeseries')

# Double check that that the time series is generates correctly
from linearsim import wave_stats
print(f'ts statistics: {wave_stats(ts,1/dt)}')

da_ts.hvplot(x='time')